In [2]:
import pandas as pd
import re

In [3]:
xlsx = pd.read_excel("noturar_geocode.xlsx")
xlsx.to_csv("noturar_raw.csv", encoding='utf-8', index=None, header=True)#, on_bad_lines='skip')
noturar_raw = pd.read_csv("noturar_raw.csv", sep=',', encoding='utf-8')
noturar_raw

,kadastr,tuman,mahalla,manzil,turi 2,turi 1,ekin maydoni,maydon,lat,lon
0,10:08:03:02:01:0187,Олмазор,Иброхим ота МФЙ,"ул. Широк, дом 100/2",NaN,часть завода,2658,367.0,69.242542,41.369780
1,10:01:03:02:02:0041,Учтепа,Кўксарой МФЙ,мас. Урикзор,NaN,территория,21670,0.0,69.123766,41.267915
2,10:01:03:02:02:0042,Учтепа,Кўксарой МФЙ,мас. Урикзор,NaN,территория,21670,0.0,69.124151,41.269373
3,10:01:03:02:02:0040,Учтепа,Кўксарой МФЙ,мас. Урикзор,NaN,территория,21670,0.0,69.124782,41.268607
4,10:01:03:02:02:0032,Учтепа,Кўксарой МФЙ,"ул. Навруз, дом 57",NaN,"магазин, производственный комплекс",183,183.0,69.126241,41.270352
...,...,...,...,...,...,...,...,...,...,...
46990,10:09:08:01:02:0006,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,"остановочный комплекс, магазин и кафе",289,288.0,69.468213,41.412033
46991,10:09:08:01:02:0012,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,магазин,69,59.0,69.468441,41.412140
46992,10:09:08:01:02:0014,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,"остановочный комплекс, магазин, столовая, дисп...",517,517.0,69.468971,41.412624
46993,10:09:08:01:02:0031,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,здание,300,103.0,69.469139,41.412641


In [4]:
noturar_raw.tuman.value_counts()

Чилонзор         8079
Мирзо Улуғбек    5167
Юнусобод         5130
Шайхонтоҳур      4789
Яшнобод          4471
Олмазор          4108
Миробод          3618
Сергели          3193
Учтепа           3129
Яккасарой        3117
Бектемир         1166
Янгихаёт         1028
Name: tuman, dtype: int64

In [5]:
noturar_raw = noturar_raw.rename(columns={'turi 2': 'turi2', 'turi 1': 'turi'})
noturar_raw['malumot'] = noturar_raw['turi']

In [59]:
def filter_types(type):
    type = re.sub(r"(\s[Nn] \d+)|(\s№\d+)|(`(.*?)`)|(\"(.*?)\")|('(.*?)')|(\s\d+.?\d*[-сонли]+ )|(ул\.\s.+)|(мас\.\s.+)|(кв\.\s.+)|(-\d+)|(№? \d+)|(\d+-)", "", type)
    return type

def replace_whole(pat, type, rep):
    if len(re.findall(pat, type)) > 0:
        return rep
    return type

In [7]:
# "часть первая (1 часть)".replace(r"\s*\([^()]*\)", "")
re.sub(r"\s*\([^()]*\)", "", "часть первая (1 часть)")

'часть первая'

In [79]:
# untouched_noturar = noturar_raw.set_index(noturar_raw.columns.difference(['turi'],sort=False).to_list())
# untouched_noturar.turi = untouched_noturar.turi.str.replace(r"(\s*\([^()]*\))", "")
# untouched_noturar = untouched_noturar.apply(lambda x: x.str.lower().str.split(r",| и |\|").explode()).reset_index()
# untouched_noturar.turi = untouched_noturar.turi.apply(lambda x: filter_types(x))

/var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/ipykernel_88097/119102816.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  untouched_noturar.turi = untouched_noturar.turi.str.replace(r"(\s*\([^()]*\))", "")


In [229]:
untouched_noturar = noturar_raw.copy()
untouched_noturar.turi = untouched_noturar.turi.str.replace(r"(\s*\([^()]*\))", "")
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(т|)о(р|)гов(о|ый|ий|ого).*б(и|ы)т(ов|).*|.*савдо.*маиший.*|.*кўп(.*)тармо.*', u'торговый комплекс, бытовой комплекс')
untouched_noturar = untouched_noturar.set_index(noturar_raw.columns.difference(['turi'],sort=False).to_list())
untouched_noturar = untouched_noturar.apply(lambda x: x.str.lower().str.split(r",| хамда | ва | и |\|").explode()).reset_index()
untouched_noturar.turi = untouched_noturar.turi.apply(lambda x: filter_types(x))
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'(\s?.?\d+.?(-|\s?))', u'')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(мини(.*)маркет|маг(.*)з(х|а|)и(н|)|д(ў|у)кон|хозтов|лар(е|ё)к).*', u'магазин')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(цех|промз).*', u'цех')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((авто|)(ст(a|о|а|)ян|турар|гара(ж|д)|парково|автопарк|са(қ|к)ла)|предп|автохоз|т(.)хташ(.?)жой|атп).*', u'автопаркинг')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(пар(е|и)(х|к)мах|(красо|г(у|ў)зал)|к(р|о)(а|о)с(.?)т|космети).*', u'парикмахерская')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(интернет|ко(ь|м|)(п|)(ь|ъ|)ютерн(ы)(й|)|комп\.|комп(.*)ют(е|о)р|ксеро|пайнет).*', u'компьютерный зал')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(кафе|о(в|а)(к|қ)атл|рестор|о(б|)(щ|ш)(е|и)(й|п)|ошхона|ч(о|а)йх(о|а)на|т(а|о)лов(ая|ой)|банкет|кухн|питани|ахвах|с(о|а)мс|шашлыч|туйх|лагман|кофе).*', u'точка общественного питания')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((т|)(а|о)(р|)гов(о|ый|ий|ого)|(с|c)авдо|торг(.*)(комп|)|(ги(.?)|су)пермаркет|коммер).*', u'торговый комплекс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*б(и|ы)т(ов|).*|.*маиший.*', u'бытовой комплекс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(банк|^((?!авиа).)*касс).*', u'банк')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(апт(е|e)к|дор(.?)хона).*', u'аптека')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(тури).*', u'туризм')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((а|o|о)с(та|)(.)о(в|ка)|бекат).*', u'остановочный комплекс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ад(и|)мин|маъмурий|государ|фу(.*)аро(.*)абул(.*)илиш|бош(.)ар|отделе|управ|абк|тамож|дав(.*)хиз|одно(.*)окн|минист|хал(.*)абул|казнач|гос(.*)услу).*', u'административное учреждение')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ма(х|ҳ)ал|гузар).*', u'махаллинский гузар')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(лаб(о|а)|диаг).*', u'лаборатория')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(клиник|мед(и|л|\.|)|боль|лечеб|массаж|реаб|хирург|(с|c)т(о|а)м(а|а|о)толог|тибби|гинеколог|узи|ветерин|лор|санитар|стацио|госпит|вакцина|к(а|о)сметоло|касалх|инфекц|шифох|давол|терапев|ортопед|санэпид|здравоох|саломат|ренген|амбулат|офтальмо|оптика).*', u'медицинское учреждение')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((^((?!авто).)*с(.?)орт)|зд(о|р)(о|р)(.?)в|со(.)лом|баня|сауна|теннис|трена|бассей|соглам|шахмат|фитн|спасалон|санатор|гимнаст|тяж(.*)атлет).*', u'спортивно-оздоровительный комплекс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(г(о|а)стин(.?)и|б(щ|ш)(е|и)жит|отел|мехмон|(ё|е)то(қ|к)|хостел).*', u'гостиница')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(с(к|)(а|)лад|баз|омбор|хранил).*', u'база')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(мтм|mtm|мактабга|maktabga|(дет.*сад)|дошк|ясли|(?<!тру)доу|бола(.*)бо).*', u'дошкольное образовательное учреждение')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(^((?!до).)*(школ|образо|та(ъ|)лим|мактаб|maktab)).*', u'средняя школа')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(м(е|и|э)чет|масжид|masjid).*', u'мечеть')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(кладбище|мавзол).*', u'кладбище')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(уче(б|ь)|ў(қ|к)ув|уечбн|пов(.*)квали|магистр|изуч).*', u'учебный комплекс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(инсти|унив|вуз|науч|нпц|микробио|исслед|академ).*', u'институт')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(т(ч|)сж|жэк).*', u'тчсж')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(музе|планетари|мемориал|городище).*', u'музей')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((?<!лом)бар|клуб|дискот).*', u'клуб-бар')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(к(о|а)(.?)дит).*', u'кондитерская')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(во(и|е)н|войск).*', u'военная часть')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(мусор|вторсыр).*', u'пункт сбора мусора')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(диспе(т|)чер).*', u'диспетчерская')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ровд|рувд|гувд|увд|ички(.*)ишл).*', u'отдел внутренних дел')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(произв|фабри|завод|комбинат|ишл|қадоқлаш|махсулот|изготов|(дока(.?)хлеб)|перераб).*', u'производственное учреждение')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(милиц|опорны).*', u'милицейский пункт')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((?<!ми)лице|колле).*', u'колледж')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((?<!го)суд).*', u'суд')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(загс|торжес|бахт(.*)уй|брако).*', u'загс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ж\/д|железн(.*)доро).*', u'железная дорога')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(кинотеатр).*', u'кинотеатр')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((?<!кино)фото).*', u'фотостудия')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((?<!кино)театр).*', u'театр')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((?<!авто)парк|отдых|сад|сайил|истирох|боғ).*', u'парк')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((авто|)(мойк|ювиш|моеч)).*', u'автомойка')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(авто(стан|вокз)).*', u'автовокзал')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((бизне|ковор)(.*)(цент|тадбир|ширка|)).*', u'бизнес центр')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(тепли|исси(.*)хона).*', u'теплица')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(пекар|лепешеч|тандир|нонвой|хлебопро|нон).*', u'пекарня')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ко(.*)дит|қандол).*', u'кондитерская')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(диспансер).*', u'диспансер')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(род(.*)(дом|компл)).*', u'родильный дом')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(дет(.*)дом|дом(.*)милосе|интернат).*', u'детский дом')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(гэс).*', u'гэс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(атс).*', u'атс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(рэс).*', u'рэс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ферм|подсоб|сел(.?)ск|скотн|сельхо|молхо|сельск).*', u'фермерское хозяйство')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(прокат).*', u'прокат')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((поли|типо)граф).*', u'типография')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(туалет|хожат).*', u'туалет')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(рсу|укс|строи|проекти|конструкт).*', u'ремонтно строительное управление')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(книжн|библио|кутуб).*', u'библиотека')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(храм|ерков|сина|монаст|часовн).*', u'храм')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(посол(.*)ств|посла).*', u'посольство')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(н(а|о)т(а|о)р).*', u'нотариус')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(адв(.)кат|консульт).*', u'консультация')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(прокурат).*', u'прокуратура')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(дворец|кул(.*)т|искус|модельн|киностудия|музыкаль|памятник|моды|творч|концерт|монумент|мад(.*)хорд|мадани|выстав(.*)зал|театр|санъат).*', u'центр культуры')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(развл|детск(ая|ой)|игров|бил(.*)яр).*', u'развлекательный комплекс')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ён(ғ|г)ин|пожарн).*', u'пожарная станция')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(молод(.)ж|дет(.*)центр|питомн|социал|ижтимо|содейст).*', u'социальный центр')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(подстан|насос|трансформ).*', u'подстанция')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(ма(с|)терск|ател(.?)е|тикув|авто(.*)(тех|услу|серв|обсл)|(?<!уча)сто|хизмат(.*)(машина|к(у|ў)рсат)|вулканиз|ремонт|сервис|тех(.*)(хизм|обсл|осмотр)|устахон|та(.*)мирлаш|бокс|зам(.*)мас|пошив|швейн|авар(.*)восста|тюнинг|портьер).*', u'мастерская')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(свадеб|плать).*', u'салон свадебных платьев')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(авто(.*)газ|азс|гзс|запра|(ё|е)(.)ил(.)и|агнкс|газо(напол|рас)|а(ё|е)(к|қ)ш).*', u'заправочная станция')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(офис|контора|филиал|идора|студия|представит).*', u'офис')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(подв(а|о)л|ерт(ў|у)).*', u'подвал')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(павил(.*)он).*', u'павильон')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(логист).*', u'центр логистики')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(тепло(?!воз)|(?<!развле)к(.)тел|озонхона).*', u'тепловой модуль')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(блок|здани|л(и|а)тер|соор|помещени|бино|иншо(о|а)т|корпус|коттедж|строени).*', u'здание')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((мо(р|т|р|-| |)*)яр(м|и)арка|рын(ок|)).*', u'ярмарка')
untouched_noturar.turi = untouched_noturar.turi.apply(lambda x: replace_whole('ярмарка', x, 'ярмарка'))
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(усл(у|и)г|кўп(.*)тарм|маи(.*)хиз).*', u'центр услуг')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*((компл|этаж|жил|много(кв|эт))(.*)(д(о|а)м|кварт|компл|масс|застр)|апарт(.*)мажм|этаж).*', u'многоэтажный жилой дом')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(пр\.|daha).*', u'проспект')
untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(тер(р|)(р|)ит|терет|сектор|участ|уч-к|хоз(.*)двор).*', u'территория')
untouched_noturar.turi = untouched_noturar.turi.apply(lambda x: x.strip())
untouched_noturar.turi = untouched_noturar.turi.apply(lambda x: "" if len(x) < 3 else x)

/var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/ipykernel_88097/4101999431.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  untouched_noturar.turi = untouched_noturar.turi.str.replace(r"(\s*\([^()]*\))", "")
/var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/ipykernel_88097/4101999431.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  untouched_noturar.turi = untouched_noturar.turi.str.replace(u'.*(т|)о(р|)гов(о|ый|ий|ого).*б(и|ы)т(ов|).*|.*савдо.*маиший.*|.*кўп(.*)тармо.*', u'торговый комплекс, бытовой комплекс')
/var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/ipykernel_88097/4101999431.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  untouched_noturar.turi = untouched_noturar.turi.str.replace(u'(\s?.?\d+.?(-|\s?))', u'')
/var/folders/ww/67t1k6ss6bqfd8tw6ntmpfgm0000gn/T/ipykernel_88097/4101999431.py:8: FutureWarning: The default v

In [236]:
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Чилонзор']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Мирзо Улуғбек']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Юнусобод']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Шайхонтоҳур']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Яшнобод']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Олмазор']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Миробод']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Сергели']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Учтепа']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Яккасарой']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Бектемир']
# yangi = untouched_noturar[untouched_noturar['tuman'] == 'Янгихаёт']
yangi = untouched_noturar
yangi.drop_duplicates(keep = False, inplace = True)
# yangi.turi.value_counts()
dict = yangi.turi.value_counts().to_dict()
yangi.turi = yangi.turi.apply(lambda x: "другое" if dict[x] < 5 or x == "" else x)
counts = yangi.turi.value_counts().reset_index()
counts.to_excel('counts.xlsx')
yangi

,kadastr,tuman,mahalla,manzil,turi2,ekin maydoni,maydon,lat,lon,turi1,turi
0,10:08:03:02:01:0187,Олмазор,Иброхим ота МФЙ,"ул. Широк, дом 100/2",NaN,2658,367.0,69.242542,41.369780,часть завода,производственное учреждение
1,10:01:03:02:02:0041,Учтепа,Кўксарой МФЙ,мас. Урикзор,NaN,21670,0.0,69.123766,41.267915,территория,территория
2,10:01:03:02:02:0042,Учтепа,Кўксарой МФЙ,мас. Урикзор,NaN,21670,0.0,69.124151,41.269373,территория,территория
3,10:01:03:02:02:0040,Учтепа,Кўксарой МФЙ,мас. Урикзор,NaN,21670,0.0,69.124782,41.268607,территория,территория
4,10:01:03:02:02:0032,Учтепа,Кўксарой МФЙ,"ул. Навруз, дом 57",NaN,183,183.0,69.126241,41.270352,"магазин, производственный комплекс",магазин
...,...,...,...,...,...,...,...,...,...,...,...
68404,10:09:08:01:02:0014,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,517,517.0,69.468971,41.412624,"остановочный комплекс, магазин, столовая, дисп...",магазин
68405,10:09:08:01:02:0014,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,517,517.0,69.468971,41.412624,"остановочный комплекс, магазин, столовая, дисп...",точка общественного питания
68406,10:09:08:01:02:0014,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,517,517.0,69.468971,41.412624,"остановочный комплекс, магазин, столовая, дисп...",диспетчерская
68407,10:09:08:01:02:0031,Мирзо Улуғбек,Улуғбек МФЙ,поселок Улугбек,NaN,300,103.0,69.469139,41.412641,здание,здание


In [ ]:
yangi.to_csv('dataframe.csv', sep=';')

In [235]:
yangi.to_excel('dataframe.xlsx')

In [187]:
str = "чф "
str.strip()

'чф'

In [198]:
dict = {"myone": 23, "mytwo": 2}
dict["myone"]

23